In [ ]:
import os
import torch
import cv2
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import kornia as K
import kornia.feature as KF
import preprocessing
import validation
import plotting
import tqdm

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

input_dir = '../../data/train/' # directory of the training data

In [ ]:
# REMOVE LATER
%load_ext autoreload
%autoreload 2

In [ ]:
# for use with MacBooks
# pip install torch

# for use with AMD GPUs on Linux
# pip3 install torch --extra-index-url https://download.pytorch.org/whl/rocm5.1.1

In [ ]:
# Load all scene names from the training data
all_scenes = preprocessing.get_scenes(input_dir)
all_scenes 

In [ ]:
# Load all image pairs, along with their covisibility and fundamental matrix

#scenes = all_scenes # load all scenes
scenes = [all_scenes[0]] # load only the first scene

pairs = preprocessing.load_pairs(scenes,input_dir)

# Load either all pairs, or only withing a specific covisibility range
pairs = pairs.reset_index()
#pairs = pairs.query('covisibility > 0.95').reset_index()
#pairs = pairs.query('0.7 < covisibility < 0.8').reset_index()

pairs

In [ ]:
# Determine if a GPU is available, otherwise use CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
# Initialize LoFTR and load the outdoor weights
matcher = KF.LoFTR(pretrained='outdoor')
matcher = matcher.to(device).eval()

In [ ]:
# Run LoFTR on the image pairs loaded previously. The output is a DataFrame containing all relevant data for each image pair analyzed.
scene_list = []
fund_matrix_list = []
pair_list = []
fund_matrix_eval = []
pair_eval = []
mkpts0_list = []
mkpts1_list = []
mconf_list = []

for index, row in pairs.head(3).iterrows(): # Head controls how many image pairs are analyzed
    
    split_pair = pairs.pair[index].split('-')
    img_id0 = split_pair[0]
    img_id1 = split_pair[1]
    
    img0_pth = os.path.join(input_dir, pairs.scene[index], "images", str(img_id0 + '.jpg'))
    img1_pth = os.path.join(input_dir, pairs.scene[index], "images", str(img_id1 + '.jpg'))
    img0 = preprocessing.load_torch_image(img0_pth, device)
    img1 = preprocessing.load_torch_image(img1_pth, device)
    batch = {"image0": K.color.rgb_to_grayscale(img0), 
            "image1": K.color.rgb_to_grayscale(img1)}
    
    with torch.no_grad():
        matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        mconf = batch['mconf'].cpu().numpy()
        
    F = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.2, 0.99999, 50000)
    
    scene_list.append(pairs.scene[index])
    fund_matrix_list.append(F[0])
    pair_list.append(pairs.pair[index])
    fund_matrix_eval.append(" ".join(str(num) for num in F[0].flatten().tolist()))
    pair_eval.append(";".join(["phototourism",pairs.scene[index],pairs.pair[index]]))
    mkpts0_list.append(mkpts0)
    mkpts1_list.append(mkpts1)
    mconf_list.append(mconf)
    
results = pd.DataFrame({'scene': scene_list, 'pair': pair_list, 'fund_matrix': fund_matrix_list, 
                        'mkpts0': mkpts0_list, 'mkpts1': mkpts1_list, 'mconf': mconf_list,
                        'pair_eval': pair_eval, 'fund_matrix_eval': fund_matrix_eval}) 

In [ ]:
results

In [ ]:
# Run the validation on all results and return the mean average accuracy
maa = validation.evaluate(input_dir, results.pair_eval, results.fund_matrix_eval)
print(f'mAA={maa} (n={len(results)})')

In [ ]:
# Draw matches for a single pair from the results

index = 7 # select image pair from results DataFrame
threshold = 0 # setting a confidence threshold (0 == no threshold)

df_draw = pd.DataFrame({'mkpts0': results.mkpts0[index].tolist(), 'mkpts1': results.mkpts1[index].tolist(), 'mconf': results.mconf[index].tolist()})

img0_pth = os.path.join(input_dir, results.scene[index], "images", str(results.pair[index].split('-')[0] + '.jpg'))
img1_pth = os.path.join(input_dir, results.scene[index], "images", str(results.pair[index].split('-')[1] + '.jpg'))
img0 = load_image(img0_pth)
img1 = load_image(img1_pth)

color = cm.jet(df_draw.query(f'mconf > {threshold}').mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(df_draw.query(f'mconf > {threshold}')))]
fig = plotting.make_matching_figure(img0, img1, np.array(df_draw.query(f'mconf > {threshold}').mkpts0.values.tolist()), 
                                    np.array(df_draw.query(f'mconf > {threshold}').mkpts1.values.tolist()), color, text=text, dpi=150, alpha = 0.1, lines = True)

In [ ]:
# Draw matches for a single pair from the results

index = 7 # select image pair from results DataFrame
threshold = 0 # setting a confidence threshold (0 == no threshold)

df_draw = pd.DataFrame({'mkpts0': results.mkpts0[index].tolist(), 'mkpts1': results.mkpts1[index].tolist(), 'mconf': results.mconf[index].tolist()})

img0_pth = os.path.join(input_dir, results.scene[index], "images", str(results.pair[index].split('-')[0] + '.jpg'))
img1_pth = os.path.join(input_dir, results.scene[index], "images", str(results.pair[index].split('-')[1] + '.jpg'))
img0 = load_image(img0_pth)
img1 = load_image(img1_pth)

color = cm.jet(df_draw.query(f'mconf > {threshold}').mconf)
color_trans = color * np.full(color.shape,255)
color_trans = np.delete(color_trans, -1, axis=1)
color_plotly = [f'rgb({",".join(c)})' for c in color_trans.astype(str)]
text = [
    'LoFTR',
    'Matches: {}'.format(len(df_draw.query(f'mconf > {threshold}')))]
fig = plotting.make_matching_figure_plotly(img0, img1, np.array(df_draw.query(f'mconf > {threshold}').mkpts0.values.tolist()), 
                                    np.array(df_draw.query(f'mconf > {threshold}').mkpts1.values.tolist()), color_plotly, text=text, alpha = 0.1)
fig

In [ ]:
F = cv2.findFundamentalMat(np.array(df_draw.query(f'mconf > {threshold}').mkpts0.values.tolist()), np.array(df_draw.query(f'mconf > {threshold}').mkpts1.values.tolist()), cv2.USAC_MAGSAC, 0.2, 0.99999, 50000)
single_evaluate_scene = results.scene[index]
single_evaluate_pair = results.pair[index]
single_evaluate_pair_eval = results.pair_eval[index]
aa = validation.evaluate_single(input_dir, single_evaluate_pair_eval, " ".join(str(num) for num in F[0].flatten().tolist()))
print(f'Accuracy = {aa[0]}\nAngle error (degrees) = {aa[1][single_evaluate_scene][single_evaluate_pair]}\nDistance error (meters) = {aa[2][single_evaluate_scene][single_evaluate_pair]}')

In [ ]:
from sklearn.cluster import DBSCAN
X = results.mkpts0[7]
Y = results.mkpts1[7]

In [ ]:
plt.scatter(X[:, 0], X[:, 1])

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1])

In [ ]:
clustering = DBSCAN(eps=20, min_samples=5).fit(X)
plt.scatter(X[:, 0], X[:, 1], c = clustering.labels_)